In [8]:
import sys
import subprocess

def install_if_needed(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Package names for import vs pip can differ
install_if_needed("Crypto")         # pycryptodome
install_if_needed("dateutil")       # python-dateutil
install_if_needed("pytz")           # pytz
install_if_needed("paho.mqtt.client")  # paho-mqtt


In [9]:
import struct
import csv
import os
from datetime import datetime
from Crypto.Cipher import AES
from datetime import date
from dateutil.parser import parse
import pytz
import paho.mqtt.client as mqttClient
import time
import json
import base64

In [10]:
def decryption(key: bytes, ciphertext: bytes) -> bytes:
    """
    Decrypt a 16-byte AES-128 ciphertext using ECB mode.

    Parameters:
        key (bytes): 16-byte AES key.
        ciphertext (bytes): 16-byte ciphertext.

    Returns:
        bytes: Decrypted plaintext.
    """
    assert len(key) == 16, "Key must be 16 bytes for AES-128"
    assert len(ciphertext) == 16, "Ciphertext must be 16 bytes"

    cipher = AES.new(key, AES.MODE_ECB)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext

In [17]:
#Hardcoded encryption key
key = bytes.fromhex("e7a5c3f2d48a0e3bc96117b5fdfba247")

#Subscribe to TTN
def on_connect(client, userdata, flags, rc):


    if rc == 0:

        print("Connected to broker")

        global Connected                #Use global variable
        Connected = True                #Signal connection

    else:
      
        print("Connection failed")

def on_message(client, userdata, message):
    #Old Lab 5 and 6 Code
    # print("")
    # print("Message received: "  + str(message.payload))

    # with open('myData.txt','a+') as f:
    #      f.write(str(message.payload)[2:-1]+"\n")

    #Process TTN
    print("\nMessage received")

    payload_str = message.payload.decode('utf-8')
    payload_json = json.loads(payload_str)

    frm_payload_base64 = payload_json['uplink_message']['frm_payload']
    ciphertext = base64.b64decode(frm_payload_base64)

    #Decrypt
    plaintext = decryption(key, ciphertext)

    print("Decrypted plaintext:", plaintext)

    # Extract systolic, diastolic, bpm (2 bytes each)
    systolic = int.from_bytes(plaintext[0:2], byteorder='big')
    diastolic = int.from_bytes(plaintext[2:4], byteorder='big')
    bpm = int.from_bytes(plaintext[4:6], byteorder='big')

    print(f"Systolic: {systolic}, Diastolic: {diastolic}, BPM: {bpm}")

     # Prepare CSV logging
    timestamp = datetime.now().isoformat()

    file_exists = os.path.isfile('bp_log.csv')
    
    # Write the new reading to CSV
    with open('bp_log.csv', mode='a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=["time", "systolic_pressure", "diastolic_pressure", "bpm"])

        # If the file is new, write the header
        if not (file_exists):
            writer.writeheader()
        
        writer.writerow({
        "time": timestamp,
        "systolic_pressure": systolic,
        "diastolic_pressure": diastolic,
        "bpm": bpm
        })

Connected = False   #global variable for the state of the connection

broker_address= "nam1.cloud.thethings.network"  #host
port = 1883                         #Broker port
user = "mae4220-telehealth@ttn" #Connection username
password = "NNSXS.5I5BKPVDVCVBZLXYKADNM47PPGYYKGAD5XGGP3I.RNGEWQB3N6Y7R3QCIWDZYRZFC5KCR6Q32VECNUXXHT5KX4TFIB3A" #<--  Put your TTN V3 API key in quotes     #Connection password

client = mqttClient.Client("Python")               #create new instance
client.username_pw_set(user, password=password)    #set username and password
client.on_connect= on_connect                      #attach function to callback
client.on_message= on_message                      #attach function to callback
client.connect(broker_address,port,60) #connect
client.subscribe(f"v3/{user}/devices/+/up") #subscribe
client.loop_forever() #then keep listening forever

Connected to broker

Message received


AssertionError: Ciphertext must be 16 bytes

In [6]:
# Test example for decryption function
if __name__ == "__main__":
    # Test vector from NIST SP 800-38A
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    ciphertext = bytes.fromhex("3ad77bb40d7a3660a89ecaf32466ef97")
    expected_plaintext = bytes.fromhex("6bc1bee22e409f96e93d7e117393172a")

    decrypted = decryption(key, ciphertext)

    print("Decrypted:", decrypted.hex())
    print("Success:", decrypted == expected_plaintext)

Decrypted: 6bc1bee22e409f96e93d7e117393172a
Success: True


In [14]:
if __name__ == "__main__":
    # Key from NIST test vector
    key = bytes.fromhex("e7a5c3f2d48a0e3bc96117b5fdfba247")

    # Your ciphertext
    ciphertext = bytes([
        0x3b, 0x73, 0x0f, 0x8e,
        0x11, 0x84, 0xa0, 0x19,
        0xa1, 0x46, 0xba, 0x83,
        0x71, 0xa6, 0x77, 0x03
    ])

    # Dummy expected plaintext for now (replace if known)
    expected_plaintext = b""  # Or put expected plaintext if you know it

    decrypted = decryption(key, ciphertext)

    print("Decrypted:", decrypted.hex())
    print("Success:", decrypted == expected_plaintext)


Decrypted: 0093005d006800000000000000000000
Success: False


In [3]:
# #Old code
# # Log file path
# log_file = 'bp_log.csv'

# # For systolic=110, diastolic=65, bpm=81 Sample data
# byte_data = struct.pack('>HHH', 110, 65, 81)


# # Unpack 3 unsigned shorts (2 bytes each, big-endian)
# systolic, diastolic, bpm = struct.unpack('>HHH', byte_data)

# # Create new reading entry
# entry = {
#     "time": datetime.utcnow().isoformat() + "Z",
#     "systolic_pressure": systolic,
#     "diastolic_pressure": diastolic,
#     "bpm": bpm
# }

# # Check if the log file exists, if not, create it with headers
# file_exists = os.path.exists(log_file)

# # Write the new reading to CSV
# with open(log_file, mode='a', newline='') as f:
#     writer = csv.DictWriter(f, fieldnames=["time", "systolic_pressure", "diastolic_pressure", "bpm"])

#     # If the file is new, write the header
#     if not file_exists:
#         writer.writeheader()
    
#     writer.writerow(entry)

# print("New BP reading saved:", entry)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
